# モジュール導入

In [ ]:
from kaggle.competitions import nflrush
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor as RFR
from tqdm import tqdm

env = nflrush.make_env()

In [ ]:
from sklearn.model_selection import GridSearchCV,train_test_split

 # データ読み込み

In [ ]:
df = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)

In [ ]:
iter_test = env.iter_test()

# 前処理

In [ ]:
new_df=df.groupby(['PlayId','Position']).count()

In [ ]:
position_count=new_df['GameId'].unstack().fillna(0).astype(int)
position_count

In [ ]:
rusher_df=df[df['NflId']==df['NflIdRusher']]

In [ ]:
rusher_df.columns

## ダミー変数化

In [ ]:
def process(df):
    df=pd.get_dummies(df,columns=['Team','PlayDirection','OffenseFormation'])
    df=df[(df['StadiumType']!='Cloudy') & (df['StadiumType']!='Bowl')]
    df=df.drop('FieldPosition', axis=1)
    outdoor=['Outdoor', 'Outdoors','Open','Indoor, Open Roof','Outdoor Retr Roof-Open', 'Oudoor', 'Ourdoor','Retr. Roof-Open','Outdor',
       'Retr. Roof - Open', 'Domed, Open', 'Domed, open', 'Outside','Heinz Field']
    indoor=['Indoors', 'RetractableRoof', 'Indoor','Retr. Roof-Closed','Dome', 'Domed, closed','Indoor, Roof Closed', 'Retr. Roof Closed','Closed Dome','Dome, closed','Domed']
    df['stadiumtype']=(df['StadiumType'].isin(outdoor)*1)
    rain=['Light Rain', 'Showers','Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.','Rain', 'Heavy lake effect snow','Snow', 'Cloudy, Rain','Rain shower','Rainy']
    df['weather']=(~df['GameWeather'].isin(rain)*1)
    df['PlayerHeight']= df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
    df['gameclock']=[ pd.Timedelta(val).total_seconds() for val in df['GameClock']]
    return df

In [ ]:
rusher_df=process(rusher_df)

In [ ]:
rusher_df=rusher_df.dropna()

In [ ]:
df_position=pd.merge(rusher_df,position_count, on='PlayId')

In [ ]:
df_position=df_position.rename(columns={'S_x':'S','S_y':'S_position'})

In [ ]:
def feature(df):
    features=pd.DataFrame(df,columns=['X', 'Y', 'S', 'A', 'Dis','Dir','YardLine', 'Quarter',
       'gameclock', 'Down', 'Distance','HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'DefendersInTheBox','PlayerHeight',
       'PlayerWeight','Temperature', 'Humidity', 'Team_home', 'stadiumtype', 'weather','PlayDirection_right',
       'OffenseFormation_ACE',
       'OffenseFormation_I_FORM', 'OffenseFormation_JUMBO',
       'OffenseFormation_PISTOL', 'OffenseFormation_SHOTGUN',
       'OffenseFormation_SINGLEBACK', 'OffenseFormation_WILDCAT', 'C', 'CB', 'DB',
       'DE', 'DL', 'DT', 'FB', 'FS', 'G', 'HB', 'ILB', 'LB', 'MLB', 'NT', 'OG',
       'OLB', 'OT', 'QB', 'RB', 'S_position', 'SAF', 'SS', 'T', 'TE', 'WR'])
    return features
    

                      

In [ ]:
features=feature(df_position)

In [ ]:
train_mean=features.mean(axis=0)

In [ ]:
train_std=features.std(axis=0)

## 正規化

In [ ]:
def normalize(features):
    X=(features-train_mean)/train_std
    return X

In [ ]:
X=normalize(features)

In [ ]:
target=pd.Series(df_position['Yards'])

In [ ]:
train_X,test_X,train_y,test_y=train_test_split(X,target,test_size=0.2)

## RandomForestで訓練

### チューニング(調整中）

In [ ]:
features.shape

In [ ]:
search_params = {
    'n_estimators'      : [300],
    'max_features'      : [3,10,20,40,'auto'],
    #'random_state'      : [1],
    #'n_jobs'            : [1],
    #'min_samples_split' : [10, 20, 30],
    'max_depth'         : [3]
}
 
gsr = GridSearchCV(
    RFR(),
    search_params,
    cv = 3,
    n_jobs = -1,
    verbose=True
)
 
gsr.fit(train_X, train_y)

In [ ]:
print(gsr.best_score_)
print(gsr.best_estimator_)
print(gsr.best_params_)

In [ ]:
def train_predict(X,target):
    clf = RFR(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
    clf.fit(X, target)
    return clf
    

In [ ]:
clf=train_predict(train_X,train_y)

In [ ]:
score_test=np.array([(i >= test_y)*1 for i in range(-99,100)])

In [ ]:
pred_y=clf.predict(test_X)

In [ ]:
score=np.array([(i >= pred_y)*1 for i in range(-99,100)])

In [ ]:
c=((score - score_test)**2).sum().sum()/(199*len(pred_y))
c

In [ ]:
train_df=df_position.iloc[:0,:]

In [ ]:
yard=['Yards' + str(i) for i in range(-99,100)]

# 予測

In [ ]:
for (test_df, sample_prediction_df) in tqdm(iter_test):
    new_df=test_df.groupby(['PlayId','Position']).count()
    position_count=new_df['GameId'].unstack().fillna(0).astype(int)
    rusher_df=test_df[test_df['NflId']==test_df['NflIdRusher']]
    test_df=process(test_df)
    test_df=pd.merge(test_df,position_count, on='PlayId')
    test_df=test_df.fillna(0)
    test_df=test_df.rename(columns={'S_x':'S','S_y':'S_position'})
    test_df=pd.concat([train_df,test_df],sort=False)
    test_df_groupby=test_df.groupby('PlayId').mean()
    test_feature=feature(test_df_groupby)
    test_feature=test_feature.fillna(0)
    test_X=normalize(test_feature)
    pred_y=clf.predict(test_X)
    pred_y=np.round(pred_y)
    score=np.array([(i >= pred_y)*1 for i in range(-99,100)])
    sample_prediction_df.iloc[0,:]=score.T
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df

In [ ]:
env.write_submission_file()

In [ ]:
import os
print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])